# Análisis de Dataset de Clientes

## 1. Instalación de librerías necesarias
Asegúrate de instalar las librerías para poder ejecutar el notebook.

In [ ]:
%pip install --upgrade pip
%pip install -r requirements.txt

## 2. Importación de librerías

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr

## 3. Carga del dataset

In [ ]:
path = "data/dataset_sintetico_clientes.xlsx"
df = pd.read_excel(path, index_col="cliente_id")
df.head(10)

### Información del DataFrame

In [ ]:
print(f"Información general del dataset:")
print(df.info())

print(f"\nValores duplicados en todo el dataset: {df.duplicated().sum()}")

print(f"\nValores NaN en todo el dataset:\n{df.isna().sum()}")

print(f"\nForma del dataset: {df.shape}")

### Resumen estadístico con medidas adicionales

In [ ]:
complete_statistic_summary = df.describe()
for col in df.columns:
    if df[col].dtype in [np.float64, np.int64]:
        complete_statistic_summary.at['median', col] = df[col].median()
        complete_statistic_summary.at['mode', col] = df[col].mode()[0]
        complete_statistic_summary.at['variance', col] = df[col].var()

complete_statistic_summary.head(12)

In [ ]:
fig, axes = plt.subplots(2,1, figsize=(4,4))
axes = axes.flatten()

sns.boxplot(x="region", y="satisfaccion", data=df, ax=axes[0])
sns.boxplot(x="categoria_producto", y="satisfaccion", data=df, ax=axes[1])

plt.tight_layout()
plt.show()



## 4. Evaluación inicial de las variables numéricas

#### Distribuciones

In [ ]:
columns = ["monto_total", "frecuencia_compra", "edad_cliente", 
           "dias_desde_ultima_compra", "descuento_aplicado", "productos_comprados"]

sns.set_style("whitegrid")
fig, axes = plt.subplots(3, 2, figsize=(4,5))
fig.suptitle("Distribución de los datos", fontsize=16)
axes = axes.flatten()
for i, col in enumerate(columns):
    sns.boxplot(data=df, y=col, ax=axes[i])
    axes[i].set_ylabel("")
    axes[i].set_xlabel(col)
plt.tight_layout()
plt.show()

#### Correlación entre variables

Primero según la satisfacción

In [ ]:
columns = ["monto_total", "frecuencia_compra", "edad_cliente", 
           "dias_desde_ultima_compra", "descuento_aplicado", "productos_comprados"]

results = []

fig, axes = plt.subplots(3, 2, figsize=(7, 9))
fig.suptitle("Regresiones de satisfacción por variables", fontsize=16)
axes = axes.flatten()

for i, col in enumerate(columns):
    sns.regplot(x=col, y="satisfaccion", data=df, ax=axes[i],
                line_kws={"color": "red", "linewidth": 2})
    
    r_p, p_p = pearsonr(df[col], df["satisfaccion"])
    r_s, p_s = spearmanr(df[col], df["satisfaccion"])

    results.append({
        "variable_analizada": col,
        "coef_pearson": r_p,
        "p_value_pearson": p_p,
        "coef_spearman": r_s,
        "p_value_spearman": p_s
    })

    axes[i].set_xlabel(col)
    axes[i].set_ylabel("Satisfacción")

plt.tight_layout()
plt.show()

corr_satisfaccion = pd.DataFrame(results)
print(corr_satisfaccion)

Después según el monto total

In [ ]:
columns = ["satisfaccion", "frecuencia_compra", "edad_cliente", 
           "dias_desde_ultima_compra", "descuento_aplicado", "productos_comprados"]

results = []

fig, axes = plt.subplots(3, 2, figsize=(7, 9))
fig.suptitle("Regresiones de monto total por variables", fontsize=16)
axes = axes.flatten()

for i, col in enumerate(columns):
    sns.regplot(x=col, y="monto_total", data=df, ax=axes[i],
                line_kws={"color": "red", "linewidth": 2})
    
    r_p, p_p = pearsonr(df[col], df["monto_total"])
    r_s, p_s = spearmanr(df[col], df["monto_total"])

    results.append({
        "variable_analizada": col,
        "coef_pearson": r_p,
        "p_value_pearson": p_p,
        "coef_spearman": r_s,
        "p_value_spearman": p_s
    })

    axes[i].set_xlabel(col)
    axes[i].set_ylabel("Monto")

plt.tight_layout()
plt.show()

corr_monto = pd.DataFrame(results)
print(corr_monto)

## 5. Feature Engineering: Creación de categorías

In [ ]:
df_clean = df.copy()

df_clean["grado_frecuencia"] = pd.qcut(
    df_clean["frecuencia_compra"],
    q=4,
    labels=["muy baja", "baja", "media", "alta"])

df_clean["tipo_gastador"] = pd.qcut(
    df_clean["monto_total"],
    q=4,
    labels=["muy bajo", "bajo", "medio", "alto"])

df_clean["cliente_reciente"] = pd.qcut(
    df_clean["dias_desde_ultima_compra"],
    q=4,
    labels=["muy reciente", "reciente", "medio reciente", "poco reciente"])

df_clean.head(10)

## 6. Agregaciones por segmento

In [ ]:
df_clean.groupby("grado_frecuencia").agg({
    "monto_total": "mean",
    "satisfaccion": "mean",
    "descuento_aplicado": "mean"
}).reset_index()

In [ ]:
df_clean.groupby("tipo_gastador").agg({
    "frecuencia_compra": "mean",
    "satisfaccion": "mean",
    "descuento_aplicado": "mean"
}).reset_index()

In [ ]:
df_clean.groupby("cliente_reciente").agg({
    "frecuencia_compra": "mean",
    "satisfaccion": "mean",
    "descuento_aplicado": "mean"
}).reset_index()

In [ ]:
df_sum = df_clean.groupby("grado_frecuencia").agg(
    monto_total_sum=("monto_total", "sum"),
    descuento_total_sum=("descuento_aplicado", "sum")
).reset_index()
df_sum

In [ ]:
monto_muy_baja = df_sum.loc[df_sum["grado_frecuencia"] == "muy baja", "monto_total_sum"].values[0]
monto_alta = df_sum.loc[df_sum["grado_frecuencia"] == "alta", "monto_total_sum"].values[0]
print(f"Diferencia entre el segmento de menor y mayor frecuencia: {monto_alta - monto_muy_baja:.2f}")

## 7. Histogramas de edad y productos comprados

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(6, 6), sharex=True)
sns.histplot(data=df_clean, x='edad_cliente', bins=30, kde=True, edgecolor='white', ax=axes[0])
axes[0].set_title('Distribución de la Edad de los Clientes')
axes[0].set_ylabel('Frecuencia')
axes[0].set_xlabel('')
for p in axes[0].patches:
    h = p.get_height()
    if h > 0:
        axes[0].annotate(f'{int(h)}', (p.get_x() + p.get_width()/2, h), ha='center', va='bottom', fontsize=8)
sns.histplot(data=df_clean, x='edad_cliente', weights='productos_comprados', bins=30, kde=True, edgecolor='white', ax=axes[1])
axes[1].set_title('Distribución de Productos Comprados por Edad')
axes[1].set_ylabel('Cantidad de Productos')
axes[1].set_xlabel('Edad')
for p in axes[1].patches:
    h = p.get_height()
    if h > 0:
        axes[1].annotate(f'{int(h)}', (p.get_x() + p.get_width()/2, h), ha='center', va='bottom', fontsize=8)
plt.show()

## 8. Evaluación de métricas por tipo de cliente

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(8, 10))
axes = axes.flatten()
color_derecha = "#4C72B0"
color_izquierda = "#8FBBD9"
fig.suptitle("Evaluación de métricas según el tipo de cliente (frecuencia y gasto)", fontsize=16)

sns.barplot(data=df_clean, x="grado_frecuencia", y="monto_total", color=color_izquierda, estimator=sum, errorbar=None, ax=axes[0])
axes[0].set_ylim(df_sum["monto_total_sum"].min()-20000, df_sum["monto_total_sum"].max()+20000)
for p in axes[0].patches:
    axes[0].annotate(f"{p.get_height():.0f}",(p.get_x() + p.get_width()/2, p.get_height()), ha="center", va="bottom")

sns.barplot(data=df_clean, x="grado_frecuencia", y="satisfaccion", color=color_izquierda, errorbar=None, ax=axes[2])
axes[2].set_ylim(2.7, 3.2)
for p in axes[2].patches:
    axes[2].annotate(f"{p.get_height():.2f}", (p.get_x() + p.get_width()/2, p.get_height()), ha="center", va="bottom")

sns.barplot(data=df_clean, x="grado_frecuencia", y="descuento_aplicado", color=color_izquierda, estimator="sum", errorbar=None, ax=axes[4])
axes[4].set_ylim(df_sum["descuento_total_sum"].min()-200, df_sum["descuento_total_sum"].max()+200)
for p in axes[4].patches:
    axes[4].annotate(f"{p.get_height():.2f}", (p.get_x() + p.get_width()/2, p.get_height()), ha="center", va="bottom")

sns.barplot(data=df_clean, x="tipo_gastador", y="frecuencia_compra", color=color_derecha, errorbar=None, ax=axes[1])
axes[1].set_ylim(23, 27)
for p in axes[1].patches:
    axes[1].annotate(f"{p.get_height():.2f}", (p.get_x() + p.get_width()/2, p.get_height()), ha="center", va="bottom")

sns.barplot(data=df_clean, x="tipo_gastador", y="satisfaccion", color=color_derecha, errorbar=None, ax=axes[3])
axes[3].set_ylim(2.7, 3.2)
for p in axes[3].patches:
    axes[3].annotate(f"{p.get_height():.2f}", (p.get_x() + p.get_width()/2, p.get_height()), ha="center", va="bottom")

sns.barplot(data=df_clean, x="tipo_gastador", y="descuento_aplicado", color=color_derecha, estimator="sum", errorbar=None, ax=axes[5])
axes[5].set_ylim(df_sum["descuento_total_sum"].min()-200, df_sum["descuento_total_sum"].max()+200)
for p in axes[5].patches:
    axes[5].annotate(f"{p.get_height():.2f}", (p.get_x() + p.get_width()/2, p.get_height()), ha="center", va="bottom")

plt.tight_layout()
plt.show()

## 9. Información por región y categoría de producto

In [ ]:
df_producto = (df_clean.groupby("categoria_producto", as_index=False)["monto_total"].sum().sort_values("monto_total", ascending=False))
df_region = (df_clean.groupby("region", as_index=False)["monto_total"].sum().sort_values("monto_total", ascending=False))

df_producto["porcentaje"] = df_producto["monto_total"] / df_producto["monto_total"].sum()
df_region["porcentaje"] = df_region["monto_total"] / df_region["monto_total"].sum()

order_producto = df_producto["categoria_producto"]
order_region = df_region["region"]

print(df_producto)
print(df_region)

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(8,8))
axes = axes.flatten()

fig.suptitle("Evaluación de métricas según el tipo de producto y la región", fontsize=16)

sns.barplot(data=df_clean, x="categoria_producto", y="monto_total", order=order_producto, estimator="sum", errorbar=None, ax=axes[0], color=color_izquierda)
for p, pct in zip(axes[0].patches, df_producto["porcentaje"]):
    h = p.get_height()
    axes[0].annotate(
        f"{int(h)}",
        (p.get_x() + p.get_width() / 2, h),
        ha='center',
        va='bottom'
    )
    axes[0].annotate(
        f"{pct:.2%}",
        (p.get_x() + p.get_width() / 2, h - 10000),
        ha='center',
        va='top',
        fontsize=8
    )
axes[0].set_title("Monto Total por Categoría de Producto")
axes[0].set_ylabel("Monto Total")
axes[0].set_ylim(500000, 750000)
axes[0].set_xlabel("")

sns.barplot(data=df_clean, x="region", y="monto_total", order=order_region, estimator="sum", errorbar=None, ax=axes[1], color = color_derecha)
for p, pct in zip(axes[1].patches, df_region["porcentaje"]):
    h = p.get_height()
    axes[1].annotate(
        f"{int(h)}",
        (p.get_x() + p.get_width() / 2, h),
        ha='center',
        va='bottom'
    )
    axes[1].annotate(
        f"{pct:.2%}",
        (p.get_x() + p.get_width() / 2, h - 10000),
        ha='center',
        va='top',
        fontsize=8, color="white"
    )
axes[1].set_title("Monto Total por Región")
axes[1].set_ylabel("Monto Total")
axes[1].set_ylim(500000, 750000)
axes[1].set_xlabel("")

sns.barplot(data=df_clean, x="categoria_producto", y="satisfaccion", order=order_producto, estimator="mean", errorbar=None, ax=axes[2], color = color_izquierda)
for p in axes[2].patches:
    h = p.get_height()
    axes[2].annotate(
        f"{h:.2f}",
        (p.get_x() + p.get_width() / 2, h),
        ha='center',
        va='bottom'
    )
axes[2].set_title("Satisfacción por Categoría de Producto")
axes[2].set_ylabel("Satisfacción")
axes[2].set_ylim(2.7, 3.2)
axes[2].set_xlabel("")

sns.barplot(data=df_clean, x="region", y="satisfaccion", order=order_region, estimator="mean", errorbar=None, ax=axes[3], color = color_derecha)
for p in axes[3].patches:
    h = p.get_height()
    axes[3].annotate(
        f"{h:.2f}",
        (p.get_x() + p.get_width() / 2, h),
        ha='center',
        va='bottom'
    )
axes[3].set_title("Satisfacción por Región")
axes[3].set_ylabel("Satisfacción")
axes[3].set_ylim(2.7, 3.2)
axes[3].set_xlabel("")

sns.boxplot(data=df_clean, x="categoria_producto", y="monto_total", ax=axes[4], color = color_izquierda, order=order_producto)
axes[4].set_title("Distribución de Monto por Categoría de Producto")
axes[4].set_xlabel("")
axes[4].set_ylabel("Monto Total")
sns.boxplot(data=df_clean, x="region", y="monto_total", ax=axes[5], color = color_derecha, order=order_region)
axes[5].set_title("Distribución de Monto por Región")
axes[5].set_xlabel("")
axes[5].set_ylabel("Monto Total")

plt.tight_layout()
plt.show()


## 10. Heatmaps de monto y satisfacción

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,5))
pivot_monto = df_clean.pivot_table(values="monto_total", index="region", columns="categoria_producto", aggfunc="sum")
pivot_satisf = df_clean.pivot_table(values="satisfaccion", index="region", columns="categoria_producto", aggfunc="mean")

sns.heatmap(pivot_monto, annot=True, fmt=".2f", cmap="YlGnBu", ax=axes[0])
axes[0].set_title("Monto Total por Región y Categoría de Producto")

sns.heatmap(pivot_satisf, annot=True, fmt=".2f", cmap="YlGnBu", ax=axes[1])
axes[1].set_title("Satisfacción Promedio por Región y Categoría de Producto")

plt.tight_layout()
plt.show()

## 11. Monto acumulado por región y producto

In [ ]:
df_sum = df_clean.groupby(["region", "categoria_producto"])["monto_total"].sum().reset_index()

monto_max = df_sum["monto_total"].max()
monto_min = df_sum["monto_total"].min()

region_min = df_sum.loc[df_sum["monto_total"] == monto_min, "region"].values[0]
producto_min = df_sum.loc[df_sum["monto_total"] == monto_min, "categoria_producto"].values[0]

region_max = df_sum.loc[df_sum["monto_total"] == monto_max, "region"].values[0]
producto_max = df_sum.loc[df_sum["monto_total"] == monto_max, "categoria_producto"].values[0]

print("Monto acumulado por región y producto:")
print(df_sum.sort_values(by="monto_total", ascending=False))
print(f"\nGasto mínimo acumulado: {monto_min:.2f} -> Región: {region_min}, Producto: {producto_min}")
print(f"Gasto máximo acumulado: {monto_max:.2f} -> Región: {region_max}, Producto: {producto_max}")
print(f"\nDiferencia entre máximo y mínimo acumulado: {monto_max - monto_min:.2f}")

## 12. Satisfacción por región y producto

In [ ]:
df_satisfaccion = df_clean.groupby(["region", "categoria_producto"])["satisfaccion"].mean().reset_index()

satisfaccion_max = df_satisfaccion["satisfaccion"].max()
satisfaccion_min = df_satisfaccion["satisfaccion"].min()

region_min = df_satisfaccion.loc[df_satisfaccion["satisfaccion"] == satisfaccion_min, "region"].values[0]
region_max = df_satisfaccion.loc[df_satisfaccion["satisfaccion"] == satisfaccion_max, "region"].values[0]

producto_min = df_satisfaccion.loc[df_satisfaccion["satisfaccion"] == satisfaccion_min, "categoria_producto"].values[0]
producto_max = df_satisfaccion.loc[df_satisfaccion["satisfaccion"] == satisfaccion_max, "categoria_producto"].values[0]

print("Satisfacción media por region y producto")
print(df_satisfaccion.sort_values("satisfaccion", ascending=False))

print(f"\nSatisfacción mínima: {satisfaccion_min:.2f} -> Región: {region_min}, Producto: {producto_min}")
print(f"Satisfacción máxima: {satisfaccion_max:.2f} -> Región: {region_max}, Producto: {producto_max}")
print(f"\nDiferencia entre máximo y mínimo: {satisfaccion_max - satisfaccion_min:.2f}")

In [ ]:
df_satisfaccion = df_clean.groupby(["region", "categoria_producto"])["satisfaccion"].mean().reset_index()

satisfaccion_max = df_satisfaccion["satisfaccion"].max()
satisfaccion_min = df_satisfaccion["satisfaccion"].min()

region_min = df_satisfaccion.loc[df_satisfaccion["satisfaccion"] == satisfaccion_min, "region"].values[0]
region_max = df_satisfaccion.loc[df_satisfaccion["satisfaccion"] == satisfaccion_max, "region"].values[0]

producto_min = df_satisfaccion.loc[df_satisfaccion["satisfaccion"] == satisfaccion_min, "categoria_producto"].values[0]
producto_max = df_satisfaccion.loc[df_satisfaccion["satisfaccion"] == satisfaccion_max, "categoria_producto"].values[0]

print("Satisfacción media por region y producto")
print(df_satisfaccion.sort_values("satisfaccion", ascending=False))

print(f"\nSatisfacción mínima: {satisfaccion_min:.2f} -> Región: {region_min}, Producto: {producto_min}")
print(f"Satisfacción máxima: {satisfaccion_max:.2f} -> Región: {region_max}, Producto: {producto_max}")
print(f"\nDiferencia entre máximo y mínimo: {diferencia:.2f}")